In [10]:
! pip install selenium webdriver_manager

DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd


In [16]:
# 드라이버 옵션 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("detach", True) # 브라우저 열어놓을지 선택

# 크롬 매니저 설치
driver = webdriver.Chrome(service = Service(options=chrome_options))


In [12]:
# 1. 네이버 지도에서 지역명으로 음식점 찾기

def get_page(local='서울 관악구'):
    driver.get('https://map.naver.com/search/' + local + '음식점') # 사이트 이동
    time.sleep(2)
    

In [13]:
# 2. iframe 사이트 이동 메서드
from selenium.common.exceptions import NoSuchElementException

def switch_ifm(name):
    try:
        driver.switch_to.frame(ifram)
        driver.switch_to.default_content() # 기존 페이지로 이동
        ifram = driver.find_element(By.ID, name) 
        print('SUCCESS : 정상적으로', name, '페이지로 이동했습니다.')
    except NameError:
        print('ERROR : 해당되는 이름을 못찾습니다.')
    except NoSuchElementException:
        print('ERROR : 웹 페이지에서 지정한 요소를 찾을 수 없습니다.')

In [14]:
# 3. 페이지 내에서 메뉴 찾기

def search_menu():
    switch_ifm('searchIframe')
    menu=[]

    driver.find_element(By.CLASS_NAME, 'XUrfU').click()
    while True :
        if len(menu) >= 60:
            break
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        menu = driver.find_elements(By.CLASS_NAME, 'TYaxT')
    print('현재 페이지에서 찾은 페이지 :', len(menu))
    return menu

In [15]:
from selenium.common.exceptions import WebDriverException     
import pandas as pd

# 4. entryIframe에서 리뷰목록으로 넘어가 댓글 수집하기

# 가게 이름, 사용자 댓글, 게시글 날짜
def search_comments():
    saler_title, user_comments, comments_dates = [], [], []
    time.sleep(2)
    switch_ifm('entryIframe') # 이동

    # 리뷰 페이지 이동
    src_name = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div')
    a=src_name.find_elements(By.TAG_NAME, 'a')
    for i in a:
        if i.text == '리뷰':
            i.click()
            break
    title = driver.find_element(By.CLASS_NAME, 'lnJFt').text
    time.sleep(1.5)
    

In [17]:
from selenium.common.exceptions import WebDriverException     
import pandas as pd

# 4. entryIframe에서 리뷰목록으로 넘어가 댓글 수집하기

# 가게 이름, 사용자 댓글, 게시글 날짜
def search_comments():
    saler_title, user_comments, comments_dates = [], [], []
    time.sleep(2)
    switch_ifm('entryIframe') # 이동

    # 리뷰 페이지 이동
    src_name = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div')
    a=src_name.find_elements(By.TAG_NAME, 'a')
    for i in a:
        if i.text == '리뷰':
            i.click()
            break
    title = driver.find_element(By.CLASS_NAME, 'lnJFt').text
    time.sleep(1.5)
    
    # 댓글 수집
    comments=driver.find_elements(By.CLASS_NAME, 'zPfVt')
    driver.execute_script("window.scrollBy(0, 500);")
    time.sleep(1)
    con = int(driver.find_element(By.CLASS_NAME, 'place_section_count').text.replace(',', ''))
    while True:
        if len(comments)>=100 or len(comments)<=1 or con < 100:
            break
        time.sleep(3)
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END) # 하단으로 이동
        lls = driver.find_element(By.CLASS_NAME, 'k1QQ5')
        try :
            driver.execute_script("window.scrollBy(0, -300);")
            lls.find_element(By.CLASS_NAME, 'fvwqf').click() # 리뷰란에서만 더보기 클릭
        except :
            print('에러')
            pass
        comments=driver.find_elements(By.CLASS_NAME, 'zPfVt')
    print('모은 댓글 갯수 =', len(comments))
    
    # 
    for j in driver.find_elements(By.CLASS_NAME, 'owAeM'):
        try:
            saler_title.append(title)
            user_comments.append(j.find_element(By.CLASS_NAME, 'zPfVt').text)
            comments_dates.append(j.find_element(By.TAG_NAME, 'time').text)

        except WebDriverException:
            print('WebDriverException에러 발생.... ')
            pass
    print(title, '수집 완료-------------------------------')
    
    result = pd.DataFrame({})
    try:
        result = pd.DataFrame({
            '가게 이름' : saler_title,
            '댓글' : user_comments,
            '작성 날짜' : comments_dates
        })
    except ValueError :
        pass
    return result

In [18]:
# 5. 합치기
li = []
locals = ['서울 관악구', '서울 강남구', '서울 압구정', '서울 청담동', '서울 이태원', '서울 홍대']
count = 0

get_page(locals[5]) # 페이지 검색
time.sleep(1.5)
menu = search_menu() # 음식점 찾기|
time.sleep(1.5)
for j in menu:
    switch_ifm('searchIframe')
    j.click()
    count += 1
    print('진행도 :', count)
    li.append(search_comments()) # 댓글 수집
    
li

SUCCESS : 정상적으로 searchIframe 페이지로 이동했습니다.
현재 페이지에서 찾은 페이지 : 60
SUCCESS : 정상적으로 searchIframe 페이지로 이동했습니다.
진행도 : 1
SUCCESS : 정상적으로 entryIframe 페이지로 이동했습니다.
모은 댓글 갯수 = 0
일식당 수집 완료-------------------------------
SUCCESS : 정상적으로 searchIframe 페이지로 이동했습니다.
진행도 : 2


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.90)
Stacktrace:
	GetHandleVerifier [0x00007FF6084D9632+30946]
	(No symbol) [0x00007FF60848E3C9]
	(No symbol) [0x00007FF608386FDA]
	(No symbol) [0x00007FF60835CB85]
	(No symbol) [0x00007FF6084037A7]
	(No symbol) [0x00007FF60841A771]
	(No symbol) [0x00007FF6083FC813]
	(No symbol) [0x00007FF6083CA6E5]
	(No symbol) [0x00007FF6083CB021]
	GetHandleVerifier [0x00007FF60860F83D+1301229]
	GetHandleVerifier [0x00007FF60861BDB7+1351783]
	GetHandleVerifier [0x00007FF608612A03+1313971]
	GetHandleVerifier [0x00007FF60850DD06+245686]
	(No symbol) [0x00007FF60849758F]
	(No symbol) [0x00007FF608493804]
	(No symbol) [0x00007FF608493992]
	(No symbol) [0x00007FF60848A3EF]
	BaseThreadInitThunk [0x00007FFDF686257D+29]
	RtlUserThreadStart [0x00007FFDF7BAAF28+40]


In [ ]:
df1 = li[0]
for i in li[1:]:
    df2 = pd.DataFrame(i)
    df1 = pd.concat([df1, df2], axis=0)

df1

,가게 이름,댓글,작성 날짜
0,이자카야,요구르트 하이볼 맛있어요!!! \n룸으로 되있어서 얘기하기도 편하고 좋습니당 ㅎㅎ ...,6.22.토
1,이자카야,대부분이 룸이라 오붓하게 술한잔하면서 대화하기 좋았어요. 안주도 다양하고 술도 다양...,5.27.월
2,이자카야,홍대 공감~♡ 맛집이라 한번 다녀오면 그 맛이 계속 생각나요.. 이번에도 주문한 음...,2.19.월
3,이자카야,룸술집이여서 좋고 오꼬노미야끼랑 명란마요 굿👍,6.15.토
4,이자카야,사시미 양이 좀 적고 회가 작아서 아쉬웠어요\n그래도 두 메뉴 다 맛있었습니다ㅎㅎ,5.16.목
...,...,...,...
3195,돼지고기구이,,23.10.30.월.월
3196,돼지고기구이,위생이 너무별로 들어가자마자 이상한 냄새가 나요 고기도 그렇게맛있는지 모르겠고 둘이...,23.9.17.일.일
3197,돼지고기구이,아 진짜 개쥰맛 너무 좋아요 맨날 가규 싶음,23.8.19.토.토
3198,돼지고기구이,고기고 맛있고 특히 전골이랑 사리는 강추합니다!!\n재방문 의사 100퍼센트에요 ㅎ...,23.9.9.토.토


In [ ]:
df1.dropna()

,가게 이름,댓글,작성 날짜
0,이자카야,요구르트 하이볼 맛있어요!!! \n룸으로 되있어서 얘기하기도 편하고 좋습니당 ㅎㅎ ...,6.22.토
1,이자카야,대부분이 룸이라 오붓하게 술한잔하면서 대화하기 좋았어요. 안주도 다양하고 술도 다양...,5.27.월
2,이자카야,홍대 공감~♡ 맛집이라 한번 다녀오면 그 맛이 계속 생각나요.. 이번에도 주문한 음...,2.19.월
3,이자카야,룸술집이여서 좋고 오꼬노미야끼랑 명란마요 굿👍,6.15.토
4,이자카야,사시미 양이 좀 적고 회가 작아서 아쉬웠어요\n그래도 두 메뉴 다 맛있었습니다ㅎㅎ,5.16.목
...,...,...,...
3195,돼지고기구이,,23.10.30.월.월
3196,돼지고기구이,위생이 너무별로 들어가자마자 이상한 냄새가 나요 고기도 그렇게맛있는지 모르겠고 둘이...,23.9.17.일.일
3197,돼지고기구이,아 진짜 개쥰맛 너무 좋아요 맨날 가규 싶음,23.8.19.토.토
3198,돼지고기구이,고기고 맛있고 특히 전골이랑 사리는 강추합니다!!\n재방문 의사 100퍼센트에요 ㅎ...,23.9.9.토.토


In [ ]:
df1.to_csv('서울_홍대_음식점_댓글수집.csv', index=False, encoding='utf-8-sig')

Error: need to escape, but no escapechar set

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38080 entries, 0 to 2439
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   가게 이름   38080 non-null  object
 1   댓글      38080 non-null  object
 2   작성 날짜   38080 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB
